In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['SERPER_API_KEY']
os.environ['LANGSMITH_API_KEY']
os.environ['LANGSMITH_PROJECT']
os.environ['TAVILY_API_KEY']
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama

In [3]:
urls=["https://www.langchain.com/langsmith","https://www.langchain.com/langgraph"]
docs=[WebBaseLoader(i).load() for i in urls]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
docs_list = [item for sublist in docs for item in sublist]
doc_splits = text_splitter.split_documents(docs_list)

# -----Add to vectorDB

llm=ChatOllama(model="llama3.2",temperature=0)

embeddings = OllamaEmbeddings(
    model="llama3.2",
)
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

# Create Documents Retrival

In [4]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import  TavilySearchResults

####---Google Search Results

search=TavilySearchResults()

##------Index Retrival
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_langchain_posts",
    "Search and return information about Langmsith or Langraph.For any question related to Langsmith or Langraph use only this tool!",
)

tools = [search,retriever_tool]
print(retriever_tool.name)
print(retriever_tool.description)

retrieve_langchain_posts
Search and return information about Langmsith or Langraph.For any question related to Langsmith or Langraph use only this tool!


In [5]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

In [6]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import  AgentExecutor
agent=create_tool_calling_agent(llm, tools, prompt)
agent_execute=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_execute.invoke({"input":"Hello What is the  Current weather in Delhi"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Current weather in Delhi'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Delhi', 'region': 'Ontario', 'country': 'Canada', 'lat': 42.85, 'lon': -80.5, 'tz_id': 'America/Toronto', 'localtime_epoch': 1737784779, 'localtime': '2025-01-25 00:59'}, 'current': {'last_updated_epoch': 1737783900, 'last_updated': '2025-01-25 00:45', 'temp_c': -9.2, 'temp_f': 15.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 238, 'wind_dir': 'WSW', 'pressure_mb': 1026.0, 'pressure_in': 30.3, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 85, 'cloud': 0, 'feelslike_c': -14.2, 'feelslike_f': 6.4, 'windchill_c': -17.6, 'windchill_f': 0.4, 'heatindex_c': -10.8, 'heatindex_f': 12.6, 'dewpoint_c': -13.0, 'dewpoint_f': 8.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, '

{'input': 'Hello What is the  Current weather in Delhi',
 'output': 'The current weather in Delhi is clear with a temperature of -9.2°C (15.4°F). The wind speed is 6.0 mph (9.7 km/h) from the west-southwest direction, and the pressure is 1026.0 mbar. There is no precipitation expected, and the humidity is at 85%.'}

In [10]:
agent_execute.invoke({"input":"What IS langraph"})



> Entering new AgentExecutor chain...

Invoking: `retrieve_langchain_posts` with `{'query': 'langraph'}`


from prototype to production.Contact UsSign UpProductsLangChainLangSmithLangGraphAgentsEvaluationRetrievalResourcesPython DocsJS/TS DocsGitHubIntegrationsChangelogCommunityLangSmith Trust PortalCompanyAboutCareersBlogTwitterLinkedInYouTubeMarketing AssetsSign up for our newsletter to stay up to dateThank you! Your submission has been received!Oops! Something went wrong while submitting the form.All systems operationalPrivacy PolicyTerms of Service

LangSmith






























Products

LangChainLangSmithLangGraphMethods

RetrievalAgentsEvaluationResources

BlogCustomer StoriesLangChain AcademyCommunityExpertsChangelogLLM Evaluations GuideState of AI AgentsBreakout Agent StoriesDocs

PythonLangChainLangSmithLangGraphJavaScriptLangChainLangSmithLangGraphCompany

performance with LangSmith.Trusted by companies taking agency in AI innovation:LangGraph helps teams of all 

{'input': 'What IS langraph',
 'output': 'LangGraph is a framework for building and scaling AI workloads, particularly conversational agents and complex task automation. It provides a flexible and robust platform for creating cognitive architectures that can handle diverse control flows and realistic scenarios.\n\nKey features of LangGraph include:\n\n1. **Flexible framework**: Supports single agent, multi-agent, hierarchical, and sequential control flows.\n2. **Robust handling of complex scenarios**: Can handle realistic and complex scenarios with ease.\n3. **Moderation and quality loops**: Allows for easy addition of moderation and quality loops to prevent agents from veering off course.\n4. **Templatization**: Enables tools, prompts, and models to be easily configured and reused.\n\nLangGraph is designed to support a wide range of applications, including:\n\n1. Conversational agents\n2. Complex task automation\n3. Custom LLM-backed experiences\n\nOverall, LangGraph provides a powerf

In [11]:
agent_execute.invoke({"input":"Hello how are you?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Hello, how are you?'}`


[{'url': 'https://www.youtube.com/watch?v=x23rTDl4AMs', 'content': "Hello! How are you? I'm fine, thank you! Our good morning song for preschool is a great emotions song to help your preschoolers learn how to express their fe"}, {'url': 'https://www.fluentu.com/blog/english/how-are-you-esl/', 'content': 'Say “hello” to as many people as you can and see how they answer your “How are you?”\nOnce you’ve mastered this question, you might be so happy that you answer\n“perfect”\nto the next person who asks how you are!\nDownload:\n This can be paired with “same old” like this:\n“How are you doing?”\n“Oh, you know… same old”\nI’ve been better\nWhen there is something wrong, you can use “I’ve been better” as your first response.\n The difference between “How are you?” and “How are you doing?”\nThese phrases are very similar, but “how are you?” is slightly more formal, and m

{'input': 'Hello how are you?',
 'output': 'Hello! How are you?\n\nThere are many ways to answer the question "How are you?" depending on your tone and relationship with the person asking. Here are a few examples:\n\n* Formal answers: "I\'m fine, thank you" or "I\'ve been better"\n* Informal answers: "I\'m good" or "I\'m doing well"\n* Creative responses: You can also use creative responses like "I\'m having a great day!" or "I\'m feeling happy and energized!"\n\nIt\'s always polite to answer the question with a positive tone, even if you\'re not feeling your best. And don\'t forget to ask how the other person is doing!\n\nIf you want to learn more about how to answer this question in English, there are many resources available online, such as videos, articles, and interactive exercises.\n\nIs there anything else I can help you with?'}

## Add Memory

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
message_history=ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_execute,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
agent_with_chat_history.invoke(
    {"input": "hi! my name is Aman how are you?"},
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'how are you today'}`


[{'url': 'https://oneminuteenglish.org/answer-how-are-you-today/', 'content': 'Learn how to respond to this common greeting in different situations, such as a conversation, an interview, or a group setting. Find out the difference between "How are you?" and "How are you today?" and see examples of both.'}, {'url': 'https://status.net/articles/examples-how-to-answer-how-are-you/', 'content': '30 Great Examples: How To Answer “How Are You?” “How are you doing?” “How have you been?” How To Answer “How Are You?” When someone asks “How are you?”, you can respond in various ways depending on your mood and situation. “I’m great, thanks for asking!” “I’m doing well, how about you?” “I’m fine, thanks.” “I’m having a rough day.” “I’m feeling a bit down.” Your response to “How are you?” can change based on who you’re talking to and where you are. “I’m doing well, thank you.” “I’

{'input': 'hi! my name is Aman how are you?',
 'chat_history': [],
 'output': "Hello Aman! I'm doing well, thank you for asking. How about you? Is there anything I can help you with today?"}

In [13]:
agent_with_chat_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...

Invoking: `retrieve_langchain_posts` with `{'query': 'Langsmith or Langraph'}`


from prototype to production.Contact UsSign UpProductsLangChainLangSmithLangGraphAgentsEvaluationRetrievalResourcesPython DocsJS/TS DocsGitHubIntegrationsChangelogCommunityLangSmith Trust PortalCompanyAboutCareersBlogTwitterLinkedInYouTubeMarketing AssetsSign up for our newsletter to stay up to dateThank you! Your submission has been received!Oops! Something went wrong while submitting the form.All systems operationalPrivacy PolicyTerms of Service

LangSmith






























Products

LangChainLangSmithLangGraphMethods

RetrievalAgentsEvaluationResources

BlogCustomer StoriesLangChain AcademyCommunityExpertsChangelogLLM Evaluations GuideState of AI AgentsBreakout Agent StoriesDocs

PythonLangChainLangSmithLangGraphJavaScriptLangChainLangSmithLangGraphCompany

performance with LangSmith.Trusted by companies taking agency in AI innovation:LangGraph helps 

{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='hi! my name is Aman how are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Aman! I'm doing well, thank you for asking. How about you? Is there anything I can help you with today?", additional_kwargs={}, response_metadata={})],
 'output': "Aman, your name is Aman. I apologize for the detour earlier! Let's get back on track. Is there anything specific you'd like to know or discuss? I'm here to help with any questions or topics you'd like to explore."}